In [1]:
import mainloc as m

In [2]:
from collections import defaultdict

In [18]:
#  bad practise
import warnings
warnings.filterwarnings('ignore')

In [3]:
PATH = {}
PATH["processed"] = "../data/amenities/processed/"

In [13]:
AMENITIES = ["supermarket", "pubs", "motorway", "library", "fuelstation"]

In [5]:
amenitiesDict = defaultdict(str)

In [136]:
# load
FOCUS = "polygons"
for i in AMENITIES:
    amenitiesDict[str(i+"-"+FOCUS)] = m.gpd.read_parquet(PATH["processed"]+ i + ".parquet")

In [166]:
# after the radius and interest is specified do the filter
# 4 sec for 5 objects. Slow. 
INTEREST = [1,1,1,1,1]
RADIUS = [1043,1200,10000,1021,3000]
FOCUS = "polygons"
rankDict = defaultdict(int)
for inte, rad, amen in zip(INTEREST, RADIUS, AMENITIES):
    if inte == 1:
        FOCUS = "polygons"
        # do the thing
        polyg = m.polygonMakerShapely(amenitiesDict[str(amen+"-"+FOCUS)], diameter=rad, numberOfPoints=30)
        boundary = m.gpd.GeoSeries(m.unary_union(polyg))
        FOCUS = "points"
        #boundRes.crs = "epsg:4326"
        amenitiesDict[str(amen+"-"+FOCUS)] = boundary
        FOCUS = "rank"
        rankDict[str(amen)] = int(len(boundary.explode(index_parts=True)))
    else:
        rankDict[str(amen)] = -1

In [167]:
FOCUS = "points"
amen = "motorway"
amenitiesDict[str(amen+"-"+FOCUS)].crs = "epsg:4326"
amenitiesDict[str(amen+"-"+FOCUS)].explore()

In [168]:
rankDict = {k: v for k, v in sorted(rankDict.items(), key=lambda item: item[1])}
rankDict

{'motorway': 2,
 'fuelstation': 462,
 'supermarket': 513,
 'library': 610,
 'pubs': 1703}

In [170]:
# do the multipolygon
r = 0
FOCUS = "points"
for amen in rankDict:
    if rankDict[amen] != - 1:
        if r == 0:
            polyg = amenitiesDict[str(amen+"-"+FOCUS)]
            boundRes = polyg.intersection(polyg)
            r += 1
            # init
        else:
            polyg = amenitiesDict[str(amen+"-"+FOCUS)]
            #display(boundRes.explore())
            boundRes = boundRes.intersection(polyg)
            r += 1
    else:
        None
boundRes.crs = "epsg:4326"
boundRes.explore()